In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

# === Step 1: Load and preprocess images ===
base_path = "PetImages"
categories = ["Cat", "Dog"]
img_size = 64
limit = 1000  # Number of images per class

data = []
labels = []

print("📥 Loading and preprocessing images...")

for category in categories:
    path = os.path.join(base_path, category)
    class_label = 0 if category == "Cat" else 1
    count = 0

    for img_name in tqdm(os.listdir(path)):
        try:
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Grayscale
            img = cv2.resize(img, (img_size, img_size))  # Resize
            img = cv2.equalizeHist(img)  # Histogram equalization

            data.append(img.flatten())  # Flatten to 1D
            labels.append(class_label)
            count += 1
            if count >= limit:
                break
        except:
            continue

print(f"✅ Total images loaded: {len(data)}")

# === Step 2: Convert to NumPy arrays ===
X = np.array(data)
y = np.array(labels)

# === Step 3: Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# === Step 4: PCA to reduce dimensions ===
print("🔄 Applying PCA...")
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print("✅ PCA completed.")

# === Step 5: Train SVM ===
print("🚀 Training SVM (Linear)...")
start = time.time()
model = LinearSVC(max_iter=2000)
model.fit(X_train, y_train)
end = time.time()
print(f"✅ SVM trained in {end - start:.2f} seconds.")

# === Step 6: Evaluate ===
y_pred = model.predict(X_test)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))
print(f"🎯 Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")


📥 Loading and preprocessing images...


  8%|██████▏                                                                      | 999/12501 [00:02<00:31, 363.37it/s]


✅ Total images loaded: 2000
🔄 Applying PCA...
✅ PCA completed.
🚀 Training SVM (Linear)...
✅ SVM trained in 0.06 seconds.

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.60      0.57       199
           1       0.56      0.50      0.53       201

    accuracy                           0.55       400
   macro avg       0.55      0.55      0.55       400
weighted avg       0.55      0.55      0.55       400

🎯 Accuracy: 55.00%
